## Analyze A/B Test Results

This project will assure you have mastered the subjects covered in the statistics lessons.  The hope is to have this project be as comprehensive of these topics as possible.  Good luck!

## Table of Contents
- [Introduction](#intro)
- [Part I - Probability](#probability)
- [Part II - A/B Test](#ab_test)
- [Part III - Regression](#regression)


<a id='intro'></a>
### Introduction

A/B tests are very commonly performed by data analysts and data scientists.  It is important that you get some practice working with the difficulties of these 

For this project, you will be working to understand the results of an A/B test run by an e-commerce website.  Your goal is to work through this notebook to help the company understand if they should implement the new page, keep the old page, or perhaps run the experiment longer to make their decision.



<a id='probability'></a>
#### Part I - Probability

To get started, let's import our libraries.

In [34]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline
#We are setting the seed to assure you get the same answers on quizzes as we set up
random.seed(42)

`1.` Now, read in the `ab_data.csv` data. Store it in `df`.  

a. Read in the dataset and take a look at the top few rows here:

In [35]:
df=pd.read_csv('ab_data.csv')
df.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


b. Use the cell below to find the number of rows in the dataset.

In [36]:
df.shape

(294478, 5)

c. The number of unique users in the dataset.

In [37]:
df['user_id'].nunique()

290584

d. The proportion of users converted.

In [38]:
df['converted'].mean()

0.11965919355605512

e. The number of times the `new_page` and `treatment` don't match.

In [39]:
x = df[(df['landing_page']=='new_page') & (df['group']=='control')].count()[1]
y = df[(df['landing_page']=='old_page') & (df['group']=='treatment')].count()[1]
total_mismatch = x+y
total_mismatch

3893

In [40]:
x=df.query(" landing_page== 'new_page' and group=='control'").count()
y=df.query("landing_page== 'old_page' and group=='treatment'").count()

total=x+y
total

user_id         3893
timestamp       3893
group           3893
landing_page    3893
converted       3893
dtype: int64

f. Do any of the rows have missing values?

In [41]:
df.isnull().sum()

user_id         0
timestamp       0
group           0
landing_page    0
converted       0
dtype: int64

`2.` For the rows where **treatment** does not match with **new_page** or **control** does not match with **old_page**, we cannot be sure if this row truly received the new or old page.

a. Now use the answer to the quiz to create a new dataset that meets the specifications from the quiz.  Store your new dataframe in **df2**.

In [42]:
df2 = df[~((df['landing_page']=='new_page') & (df['group']=='control'))]
df2 = df2[~((df['landing_page']=='old_page') & (df['group']=='treatment'))]
df2.size

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


1452925

In [43]:
# Double Check all of the correct rows were removed - this should be 0
df2[((df2['group'] == 'treatment') == (df2['landing_page'] == 'new_page')) == False].shape[0]

0

`3.` Use **df2** and the cells below to answer questions 

a. How many unique **user_id**s are in **df2**?

In [44]:
df2['user_id'].nunique()

290584

b. There is one **user_id** repeated in **df2**.  What is it?

In [45]:
df2[df2['user_id'].duplicated()].user_id

2893    773192
Name: user_id, dtype: int64

c. What is the row information for the repeat **user_id**? 

In [46]:
df2[df2['user_id'].duplicated()]

,user_id,timestamp,group,landing_page,converted
2893,773192,2017-01-14 02:55:59.590927,treatment,new_page,0


d. Remove **one** of the rows with a duplicate **user_id**, but keep your dataframe as **df2**.

In [47]:
df2=df2.drop(2893,axis=0)

In [48]:
df2[df2['user_id'].duplicated()]

,user_id,timestamp,group,landing_page,converted


`4.` Use **df2** in the cells below to answer 

a. What is the probability of an individual converting regardless of the page they receive?

In [49]:
df2['converted'].mean()

0.11959708724499628

b. Given that an individual was in the `control` group, what is the probability they converted?

In [50]:
prob1=df2.query("group=='control' and converted==1").shape[0]/df2.query("group=='control'").shape[0]
prob1

0.1203863045004612

In [51]:
prob_1=len(df2.query("group=='control' and converted==1"))/len(df2.query("group=='control'"))
prob_1

0.1203863045004612

c. Given that an individual was in the `treatment` group, what is the probability they converted?

In [52]:
prob2=df2.query("group=='treatment' and converted==1").shape[0]/df2.query("group=='treatment'").shape[0]
prob2

0.11880806551510564

In [53]:
prob_2=len(df2.query("group=='treatment' and converted==1"))/len(df2.query("group=='treatment'"))
prob_2

0.11880806551510564

d. What is the probability that an individual received the new page?

In [54]:
prob3=df2.query("landing_page=='new_page'").shape[0]/df2.shape[0]
prob3

0.5000619442226688

In [55]:
prob_3=len(df2.query("landing_page=='new_page'"))/len(df2)
prob_3

0.5000619442226688

In [56]:
(df2['landing_page']=='new_page').mean()

0.50006194422266881

e. Consider your results from parts (a) through (d) above, and explain below whether you think there is sufficient evidence to conclude that the new treatment page leads to more conversions.

**From the result in this data the probability for users that recive the new treatment and convert is less than the probability of users that on control group and converted that is mean there is no sufficient evidence to conclude that the new treatment page leads to more conversions  .**

<a id='ab_test'></a>
### Part II - A/B Test

Notice that because of the time stamp associated with each event, you could technically run a hypothesis test continuously as each observation was observed.  

However, then the hard question is do you stop as soon as one page is considered significantly better than another or does it need to happen consistently for a certain amount of time?  How long do you run to render a decision that neither page is better than another?  

These questions are the difficult parts associated with A/B tests in general.  


`1.` For now, consider you need to make the decision just based on all the data provided.  If you want to assume that the old page is better unless the new page proves to be definitely better at a Type I error rate of 5%, what should your null and alternative hypotheses be?  You can state your hypothesis in terms of words or in terms of **$p_{old}$** and **$p_{new}$**, which are the converted rates for the old and new pages.

**$$ H_0:P_{new}-P_{old} \leq 0 $$ 
$$ H_1:P_{new}-P{old} > 0 $$
**


`2.` Assume under the null hypothesis, $p_{new}$ and $p_{old}$ both have "true" success rates equal to the **converted** success rate regardless of page - that is $p_{new}$ and $p_{old}$ are equal. Furthermore, assume they are equal to the **converted** rate in **ab_data.csv** regardless of the page. <br><br>

Use a sample size for each page equal to the ones in **ab_data.csv**.  <br><br>

Perform the sampling distribution for the difference in **converted** between the two pages over 10,000 iterations of calculating an estimate from the null.  <br><br>

Use the cells below to provide the necessary parts of this simulation.  If this doesn't make complete sense right now, don't worry - you are going to work through the problems below to complete this problem.  

a. What is the **conversion rate** for $p_{new}$ under the null? 

In [57]:
p_new=len(df2.query("converted==1"))/len(df2)
p_new

0.11959708724499628

b. What is the **conversion rate** for $p_{old}$ under the null? <br><br>

In [58]:
p_old=len(df2.query("converted==1"))/len(df2)
p_old

0.11959708724499628

In [59]:
p_new-p_old

0.0

c. What is $n_{new}$, the number of individuals in the treatment group?

In [60]:
n_new=len(df2.query("group=='treatment'"))
n_new

145310

d. What is $n_{old}$, the number of individuals in the control group?

In [61]:
n_old=len(df2.query("group=='control'"))
n_old

145274

e. Simulate $n_{new}$ transactions with a conversion rate of $p_{new}$ under the null.  Store these $n_{new}$ 1's and 0's in **new_page_converted**.

In [66]:
new_page_converted=np.random.choice([0,1],size=n_new,p=[1-p_new,p_new])

f. Simulate $n_{old}$ transactions with a conversion rate of $p_{old}$ under the null.  Store these $n_{old}$ 1's and 0's in **old_page_converted**.

In [67]:
old_page_converted=np.random.choice([0,1],size=n_old,p=[1-p_old,p_old])

g. Find $p_{new}$ - $p_{old}$ for your simulated values from part (e) and (f).

In [68]:
diff_obs=(new_page_converted.sum()/n_new)-(old_page_converted.sum()/n_old)
diff_obs

-0.0011792117094033272

h. Create 10,000 $p_{new}$ - $p_{old}$ values using the same simulation process you used in parts (a) through (g) above. Store all 10,000 values in a NumPy array called **p_diffs**.

In [ ]:
#creating array contain [0,1] with proption from cleaned data 
p_diffs=[]
for i in range(10000):
    new_page_converted=np.random.choice([0,1],size=n_new,p=[1-p_new,p_new])
    old_page_converted=np.random.choice([0,1],size=n_old,p=[1-p_old,p_old])
    diff_obs=(new_page_converted.sum()/n_new)-(old_page_converted.sum()/n_old)
    p_diffs.append(diff_obs)

p_diffs=np.array(p_diffs)    
    
    

i. Plot a histogram of the **p_diffs**.  Does this plot look like what you expected?  Use the matching problem in the classroom to assure you fully understand what was computed here.

In [ ]:
plt.hist(p_diffs)


In [ ]:
Null_hypoth = np.random.normal(0, np.std(p_diffs), size=10000)

In [ ]:
p_diff_obs=df2[df2['landing_page']=='new_page']['converted'].mean()-df2[df2['landing_page']=='old_page']['converted'].mean()
p_diff_obs

In [ ]:
plt.hist(Null_hypoth)
plt.axvline(x=p_diff_obs,color="r");


j. What proportion of the **p_diffs** are greater than the actual difference observed in **ab_data.csv**?

In [ ]:
p_value=(p_diff_obs<p_diffs).mean()
p_value

k. Please explain using the vocabulary you've learned in this course what you just computed in part **j.**  What is this value called in scientific studies?  What does this value mean in terms of whether or not there is a difference between the new and old pages?

**According to P_value(0.9055), which is greater than alpha(0.05) we fall to reject the null hypothesis that is mean that the old page have the most conversion rates, in other word the new page doesn’t have better conversion rates than the old page.**

l. We could also use a built-in to achieve similar results.  Though using the built-in might be easier to code, the above portions are a walkthrough of the ideas that are critical to correctly thinking about statistical significance. Fill in the below to calculate the number of conversions for each page, as well as the number of individuals who received each page. Let `n_old` and `n_new` refer the the number of rows associated with the old page and new pages, respectively.

In [ ]:
#Using Z-test 
import statsmodels.api as sm

convert_old = len(df2.query("landing_page=='old_page' and converted==1 "))
convert_new = len(df2.query("landing_page=='new_page' and converted==1 "))
n_old = len(df2.query("landing_page=='old_page'"))
n_new = len(df2.query("landing_page=='new_page'"))

m. Now use `stats.proportions_ztest` to compute your test statistic and p-value.  [Here](https://docs.w3cub.com/statsmodels/generated/statsmodels.stats.proportion.proportions_ztest/) is a helpful link on using the built in.

In [ ]:
z_score,p_value=sm.stats.proportions_ztest([convert_new,convert_old],[n_new,n_old],alternative='larger')
z_score,p_value

n. What do the z-score and p-value you computed in the previous question mean for the conversion rates of the old and new pages?  Do they agree with the findings in parts **j.** and **k.**?

**The z_score and P_value mean that we fall to reject the null hypothesis mean for the conversion rates of the old and new pages that the new page doesn’t have better conversion rates than the old page.
This decision totally agree with the findings in parts j and k
.**

<a id='regression'></a>
### Part III - A regression approach

`1.` In this final part, you will see that the result you achieved in the A/B test in Part II above can also be achieved by performing regression.<br><br> 

a. Since each row is either a conversion or no conversion, what type of regression should you be performing in this case?

**Logistic Regression.**

b. The goal is to use **statsmodels** to fit the regression model you specified in part **a.** to see if there is a significant difference in conversion based on which page a customer receives. However, you first need to create in df2 a column for the intercept, and create a dummy variable column for which page each user received.  Add an **intercept** column, as well as an **ab_page** column, which is 1 when an individual receives the **treatment** and 0 if **control**.

In [ ]:
#Ceating the intercept and dummies columns for ab_page
df2['intercept']=1
df2[['control','ab_page']]=pd.get_dummies(df2['group'])
df2.drop(labels=['control'],axis=1,inplace=True)
df2.head()

c. Use **statsmodels** to instantiate your regression model on the two columns you created in part b., then fit the model using the two columns you created in part **b.** to predict whether or not an individual converts. 

In [ ]:
#fit Logistic Regression
import statsmodels.api as sm

logit=sm.Logit(df2['converted'],df2[['intercept','ab_page']])
res=logit.fit()

d. Provide the summary of your model below, and use it as necessary to answer the following questions.

In [ ]:
import scipy.stats as st
st.chisqprob=lambda chisq, df:st.chi2.sf(chisq,df)
print(res.summary())

e. What is the p-value associated with **ab_page**? Why does it differ from the value you found in **Part II**?<br><br>  **Hint**: What are the null and alternative hypotheses associated with your regression model, and how do they compare to the null and alternative hypotheses in **Part II**?

**p-value associated with ab_page is = 0.190 The P_value here tell that the new page is statistically insignificant as 0.19>0.05. It different from part II is in this section the hypothesis is tow tail test but in part II it was single tail test   .**

**Hypothesis in Part III:**

$$ H_0:p_{new}-p_{old}=0$$
$$ H_1:p_{new}-p_{old}\neq 0 $$

**Hypothesis in Part II:**
$$ H_0:P_{new}-P_{old} \leq 0 $$ 
$$ H_1:P_{new}-P{old} > 0 $$


f. Now, you are considering other things that might influence whether or not an individual converts.  Discuss why it is a good idea to consider other factors to add into your regression model.  Are there any disadvantages to adding additional terms into your regression model?

**There might be many factors that can effect individual converts . age,gender and interst can play a significant change. Also usage and access may effect the rate of conversion. We can find new trends using other factors but there may be some disadvantages like even with new factors we may miss some other influencing factors which lead to unreliable and contradictory results compared to previous results**

g. Now along with testing if the conversion rate changes for different pages, also add an effect based on which country a user lives in. You will need to read in the **countries.csv** dataset and merge together your datasets on the appropriate rows.  [Here](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.join.html) are the docs for joining tables. 

Does it appear that country had an impact on conversion?  Don't forget to create dummy variables for these country columns - **Hint: You will need two columns for the three dummy variables.** Provide the statistical output as well as a written response to answer this question.

In [ ]:
countries_df = pd.read_csv('./countries.csv')
df3= countries_df.set_index('user_id').join(df2.set_index('user_id'), how='inner')
df3.head()

In [ ]:
# Addin countries as dummies variables 
df3[['CA','UK','US']]=pd.get_dummies(df3['country'])
df3.head()

In [ ]:
#fit the logistic Regression with the counteries
logit2=sm.Logit(df3['converted'],df3[['intercept','ab_page','CA','US']])
res2=logit2.fit()
print(res2.summary())

**We fall to reject the Null Hypothesis as there is insignificant difference in conversion rates with adding countries in the model.**


h. Though you have now looked at the individual factors of country and page on conversion, we would now like to look at an interaction between page and country to see if there significant effects on conversion.  Create the necessary additional columns, and fit the new model.  

Provide the summary results, and your conclusions based on the results.

In [ ]:
#make Additional interactive columns
df3['US_inter'] = df3['US']*df3['ab_page']
df3['CA_inter']=df3['CA']*df3['ab_page']

In [ ]:
#fit the logistic Regression with intractive variables
logit3 = sm.Logit(df3['converted'], df3[['intercept','ab_page','US','CA','US_inter', 'CA_inter']])
res3 = logit3.fit()
print(res3.summary())


**We fall to reject the Null Hypothesis as there is insignificant difference in conversion rates with adding intractive variable in the model.**


**Conclusions**


**For practical and statistical reasons the three approach that used for analyze these data to see. If the new page will lead to more conversion rate than the old page, the statistical reasons by falling to reject the null hypotheses in hypotheses testing and reject the null hypotheses in all three logistic regression for the ab_page variable that mean the ab_page is insignificant. Therefore, no statistical reason tell the new page will lead to more conversion rate. Moreover, for practical reason we can find new trends using other factors but there may be some disadvantages as even with new factors we may miss some other influencing factors, which lead to unreliable and contradictory results.** 

In [ ]:
from subprocess import call
call(['python', '-m', 'nbconvert', 'Analyze_ab_test_results_notebook.ipynb'])